In [ ]:
!pip install flask pyngrok TTS torchaudio torch --quiet
!pip install coqui-tts
!pip install hangul-romanize==0.1.0
!pip install pyngrok
!ngrok config add-authtoken 32ExNftvCzN8284n6QW9VLON9Sc_7vZbU27FFxmcwSd96nwKH


ERROR: Ignored the following versions that require a different python version: 0.0.10.2 Requires-Python >=3.6.0, <3.9; 0.0.10.3 Requires-Python >=3.6.0, <3.9; 0.0.11 Requires-Python >=3.6.0, <3.9; 0.0.12 Requires-Python >=3.6.0, <3.9; 0.0.13.1 Requires-Python >=3.6.0, <3.9; 0.0.13.2 Requires-Python >=3.6.0, <3.9; 0.0.14.1 Requires-Python >=3.6.0, <3.9; 0.0.15 Requires-Python >=3.6.0, <3.9; 0.0.15.1 Requires-Python >=3.6.0, <3.9; 0.0.9 Requires-Python >=3.6.0, <3.9; 0.0.9.1 Requires-Python >=3.6.0, <3.9; 0.0.9.2 Requires-Python >=3.6.0, <3.9; 0.0.9a10 Requires-Python >=3.6.0, <3.9; 0.0.9a9 Requires-Python >=3.6.0, <3.9; 0.1.0 Requires-Python >=3.6.0, <3.10; 0.1.1 Requires-Python >=3.6.0, <3.10; 0.1.2 Requires-Python >=3.6.0, <3.10; 0.1.3 Requires-Python >=3.6.0, <3.10; 0.10.0 Requires-Python >=3.7.0, <3.11; 0.10.1 Requires-Python >=3.7.0, <3.11; 0.10.2 Requires-Python >=3.7.0, <3.11; 0.11.0 Requires-Python >=3.7.0, <3.11; 0.11.1 Requires-Python >=3.7.0, <3.11; 0.12.0 Requires-Python >=3

In [ ]:
# 설치
!pip -q install pyngrok==7.1.6 soundfile

# 로그인 (한 번만)
from pyngrok import ngrok, conf
conf.get_default().auth_token = "32ExNftvCzN8284n6QW9VLON9Sc_7vZbU27FFxmcwSd96nwKH"

# 터널 오픈 (Flask 5000 포트 노출)
public_url = ngrok.connect(5000, "http", bind_tls=True).public_url
print("🌐 Colab TTS 서버 주소:", public_url)


🌐 Colab TTS 서버 주소: https://9c667aa1a329.ngrok-free.app


In [ ]:
from google.colab import drive
import shutil
import os

# 1. 구글 드라이브 마운트
drive.mount('/content/drive')

# 2. 원본 모델 폴더 경로 (네 드라이브에 저장된 경로)
src = "/content/drive/MyDrive/TTS_MODELS"

# 3. 복사할 Colab 로컬 경로
dst = "/content/xtts"

# 4. 이미 있으면 삭제 후 복사 (덮어쓰기)
if os.path.exists(dst):
    shutil.rmtree(dst)   # 폴더 통째 삭제
shutil.copytree(src, dst)

print("✅ 모델이 /content/xtts 로 복사 완료!")


Mounted at /content/drive
✅ 모델이 /content/xtts 로 복사 완료!


In [ ]:
from flask import Flask, request, send_file, jsonify
import torch, io, os
import soundfile as sf
from TTS.api import TTS

app = Flask(__name__)
TTS_MODELS, SPEAKER_WAV_PATHS = {}, {}

def load_tts_models():
    base_path = "/content/xtts"
    for folder in os.listdir(base_path):
        if folder.startswith("xtts_model_"):
            speaker = folder.replace("xtts_model_", "")
            model_path = os.path.join(base_path, folder)
            config_path = os.path.join(model_path, "config.json")
            speaker_wav = os.path.join(model_path, "speaker.wav")
            if os.path.exists(config_path) and os.path.exists(speaker_wav):
                print(f"🔄 {speaker} 로드 중...")
                tts = TTS(model_path=model_path, config_path=config_path, gpu=True)
                TTS_MODELS[speaker] = tts
                SPEAKER_WAV_PATHS[speaker] = speaker_wav
                print(f"✅ {speaker} 모델 로드 완료")

@app.before_request
def _log_req():
    print(f"[REQ] {request.method} {request.path} from {request.remote_addr}")
    if request.is_json:
        print(f"[REQ] JSON: {request.get_json(silent=True)}")

@app.after_request
def _log_resp(resp):
    print(f"[RESP] {resp.status} {request.path}")
    return resp

@app.route("/", methods=["GET"])
def index():
    return jsonify({"status": "ok", "voices": list(TTS_MODELS.keys())})

# 루트(/)와 /tts 둘 다 허용해 혼동 제거
@app.route("/", methods=["POST"])
@app.route("/tts", methods=["POST"])
def tts_api():
    try:
        data = request.get_json(silent=True) or {}
        text = data.get("text")
        speaker = data.get("speaker")
        print(f"[DEBUG] text={text}, speaker={speaker}")

        if not text or not speaker:
            return jsonify({"error": "텍스트/스피커 누락"}), 400
        if speaker not in TTS_MODELS:
            return jsonify({"error": f"알 수 없는 화자: {speaker}"}), 400

        tts = TTS_MODELS[speaker]
        speaker_wav = SPEAKER_WAV_PATHS[speaker]
        wav = tts.tts(text=text, speaker_wav=speaker_wav, language="ko")
        if isinstance(wav, torch.Tensor):
            wav = wav.detach().cpu().numpy()

        print(f"[DEBUG] Generated wav length: {len(wav)}")

        audio_data = io.BytesIO()
        sf.write(audio_data, wav, 24000, format="WAV")
        audio_data.seek(0)
        return send_file(audio_data, mimetype="audio/wav")
    except Exception as e:
        print(f"[ERROR] {e}")
        return jsonify({"error": str(e)}), 500



/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


In [ ]:
if __name__ == "__main__":
    load_tts_models()
    app.run(port=5000, debug=False)


🔄 sy 로드 중...


/usr/local/lib/python3.12/dist-packages/TTS/api.py:93: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


✅ sy 모델 로드 완료
🔄 jung 로드 중...
✅ jung 모델 로드 완료
🔄 yj 로드 중...
✅ yj 모델 로드 완료
🔄 iu 로드 중...
✅ iu 모델 로드 완료
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


[REQ] POST / from 127.0.0.1
[REQ] JSON: {'text': '노란색 번개 모양이네. 이 사진은 오늘 있었던 어떤 일과 관련이 있니? 자세히 이야기해줄 수 있어?', 'speaker': 'sy'}
[DEBUG] text=노란색 번개 모양이네. 이 사진은 오늘 있었던 어떤 일과 관련이 있니? 자세히 이야기해줄 수 있어?, speaker=sy
[REQ] POST / from 127.0.0.1
[REQ] JSON: {'text': '노란색 번개 모양이네. 이 사진은 오늘 있었던 어떤 일과 관련이 있니? 자세히 이야기해줄 수 있어?', 'speaker': 'sy'}
[DEBUG] text=노란색 번개 모양이네. 이 사진은 오늘 있었던 어떤 일과 관련이 있니? 자세히 이야기해줄 수 있어?, speaker=sy


...

[DEBUG] Generated wav length: 193328
[RESP] 200 OK /
[REQ] POST / from 127.0.0.1
[REQ] JSON: {'text': '와, 이건 게임 화면인데 마스터 랭크 승급했네? 진짜 축하한다! 이 사진은 오늘 있었던 어떤 일과 관련이 있니? 자세히 이야기해줄 수 있어?', 'speaker': 'sy'}
[DEBUG] text=와, 이건 게임 화면인데 마스터 랭크 승급했네? 진짜 축하한다! 이 사진은 오늘 있었던 어떤 일과 관련이 있니? 자세히 이야기해줄 수 있어?, speaker=sy


INFO:werkzeug:127.0.0.1 - - [27/Oct/2025 05:28:40] "POST / HTTP/1.1" 200 -


[DEBUG] Generated wav length: 274752
[RESP] 200 OK /
[REQ] POST / from 127.0.0.1
[REQ] JSON: {'text': '사진 이야기는 이제 끝났구나. 그럼 오늘 하루 어땠는지 궁금하다! 혹시 오늘 하루를 대표하는 장면이 있었다면 어떤 거야?', 'speaker': 'sy'}
[DEBUG] text=사진 이야기는 이제 끝났구나. 그럼 오늘 하루 어땠는지 궁금하다! 혹시 오늘 하루를 대표하는 장면이 있었다면 어떤 거야?, speaker=sy


INFO:werkzeug:127.0.0.1 - - [27/Oct/2025 05:29:08] "POST / HTTP/1.1" 200 -


[DEBUG] Generated wav length: 225840
[RESP] 200 OK /
